Lets do few shot inference against our EHR FHIR PostGre SQL database



1.   Load and test Open AI access
2.   Load and test EHR PostgreSQL database access
3.   Load FAISS









# 0. Mount drive and define paths

# Installs

In [1]:
%%capture
# Clean Install/Upgrade for ALL Core LLM/QLoRA Libraries
# -U ensures the latest versions (best compatibility)
# -q suppresses the long output
!pip install -U -q accelerate peft transformers trl bitsandbytes sentencepiece

In [2]:
%%capture
# --- 1) Install libraries (once per runtime) ---
!pip -q install --upgrade openai langchain langchain-community langchain-openai faiss-cpu

In [3]:
# --- 0) Mount Drive and set your variables ---
from google.colab import drive
drive.mount('/content/drive')

DEV_PATH = "/content/drive/MyDrive/210_Capstone/210_Factory/210_dev"
FAISS_DB_PATH = DEV_PATH + "/vectorstores/medintellagent_faiss_v1"
POSTGRES_DB_PATH = DEV_PATH + "/synthea_ehr_backup.sql"  # (dump file; not used in this snippet)
LLM_MODEL = "gpt-4o-mini"
EMBEDDING_MODEL = "text-embedding-3-large"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# --- 2) Load your OpenAI key (recommended: Colab "Secrets" → OPENAI_API_KEY) ---
import os
try:
    from google.colab import userdata
    key = userdata.get('OPENAI_API_KEY')
    if key: os.environ['OPENAI_API_KEY'] = key
except Exception:
    pass

if not os.environ.get("OPENAI_API_KEY"):
    import getpass
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI_API_KEY: ")

# Initialize OpenAI client
from openai import OpenAI
client = OpenAI()


In [5]:
# --- 3) Load  FAISS vector store (must use same embedding model used to build it) ---
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vs = FAISS.load_local(FAISS_DB_PATH, embeddings, allow_dangerous_deserialization=True)

print("FAISS loaded. Example count:", len(vs.docstore._dict))


FAISS loaded. Example count: 90


In [6]:
# --- Prompt building helpers ---

# Keep this aligned to your actual DB schema + rules
PREFIX = (
    "Return a single PostgreSQL SELECT only.\n"
    "Use only tables: patients, encounters, conditions, observations, medication_requests, procedures.\n"
    "Use only parameter :patient_id. Prefer DISTINCT ON with ORDER BY for 'latest per X'; no CTEs or window functions.\n"
    "Do not mix GROUP BY with DISTINCT ON. If aggregation is needed (e.g., pairing BP), use GROUP BY + MAX(CASE...).\n"
    "Important: medication_requests has no rxnorm_code (use med_name only). Encounters has no location.\n"
    "If the question mentions 'blood pressure' or 'BP', return only systolic (8480-6) and diastolic (8462-4) results and prefer paired rows grouped by effective_datetime.\n"
    "\n"
    "Schema hints:\n"
    "  conditions(display, code, onset_datetime, abatement_datetime, encounter_id, patient_id, condition_id)\n"
    "  observations(display, loinc_code, value_num, value_unit, effective_datetime, encounter_id, patient_id, observation_id)\n"
    "  medication_requests(med_name, dose, route, start_datetime, end_datetime, refills, encounter_id, patient_id, med_request_id)\n"
    "  encounters(start_datetime, end_datetime, reason_text, class, encounter_id, patient_id)\n"
    "  procedures(display, code, performed_datetime, encounter_id, patient_id, procedure_id)\n"
    "Output only the raw SQL, no markdown fences."
)

def get_few_shots(query: str, k: int = 3):
    # uses your already-loaded FAISS vector store: `vs`
    docs_scores = vs.similarity_search_with_score(query, k=k)
    examples = [{"question": d.page_content, "sql": (d.metadata or {}).get("sql","")} for d, _ in docs_scores]
    return examples

def format_examples(examples):
    return "\n".join([f"Question: {ex['question']}\nSQL:\n{ex['sql']}\n" for ex in examples])

def build_prompt(user_question: str, k: int = 3) -> str:
    examples = get_few_shots(user_question, k=k)
    return f"{PREFIX}\n{format_examples(examples)}\nQuestion: {user_question}\nSQL:"


In [7]:
# --- 5) SQL generation + a tiny safety check ---
import re

SELECT_ONLY = re.compile(r"^\s*select\b", re.IGNORECASE | re.DOTALL)

def clean_sql(text: str) -> str:
    s = text.strip()

    # strip a leading "SQL:" line if present
    if s.lower().startswith("sql:"):
        s = s[4:].strip()

    # strip fenced code blocks like ```sql ... ``` or ``` ... ```
    m = re.match(r"^```(?:\s*sql)?\s*([\s\S]*?)\s*```$", s, flags=re.IGNORECASE)
    if m:
        s = m.group(1).strip()

    # strip stray backticks if the model emitted them oddly
    if s.startswith("```") and "```" in s[3:]:
        s = s.split("```", 1)[1].rsplit("```", 1)[0].strip()

    # remove BOM or weird invisibles
    s = s.replace("\ufeff", "").replace("\u200b", "").strip()
    return s

def is_safe_select(text: str) -> bool:
    sql = clean_sql(text)

    # (optional) reject multiple statements (allow a single trailing semicolon)
    trimmed = sql.strip()
    if ";" in trimmed[:-1]:  # semicolon before the last char
        return False

    if not SELECT_ONLY.match(trimmed):
        return False

    banned = (" insert ", " update ", " delete ", " drop ", " alter ",
              " create ", " grant ", " revoke ", " truncate ")
    low = f" {trimmed.lower()} "  # pad with spaces to avoid substring accidents
    return not any(b in low for b in banned)

def generate_sql(user_question: str, k: int = 3, max_tokens: int = 400):
    prompt = build_prompt(user_question, k=k)
    resp = client.chat.completions.create(
        model=LLM_MODEL,
        temperature=0,
        messages=[
            {"role":"system","content":"You are a precise SQL generator for a patient portal."},
            {"role":"user","content": prompt}
        ],
        max_tokens=max_tokens,
    )
    sql = resp.choices[0].message.content.strip()
    return sql

# Demo
demo_q = "Which medications am I currently taking?"
sql = generate_sql(demo_q, k=3)
print(sql, "\n\nSAFE:", is_safe_select(sql))


SELECT DISTINCT ON (mr.patient_id, mr.med_name)
  mr.patient_id,
  mr.med_name AS medication,
  mr.dose,
  mr.route,
  mr.start_datetime,
  mr.end_datetime,
  mr.refills
FROM medication_requests mr
WHERE mr.patient_id = :patient_id
  AND (mr.end_datetime IS NULL OR mr.end_datetime >= NOW())
ORDER BY mr.patient_id,
         mr.med_name,
         COALESCE(mr.end_datetime, mr.start_datetime) DESC NULLS LAST; 

SAFE: True


# Load PostgreSQL EHR FHIR Database

In [8]:
%%capture
!apt-get -y update
!apt-get -y install postgresql postgresql-contrib

!service postgresql start
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres createdb synthea_ehr

!echo "PostgreSQL installed, service started, user password set to 'postgres', and DB 'synthea_ehr' created."


In [9]:
import subprocess
import os

# Database connection details
DB_NAME = "synthea_ehr"
DB_USER = "postgres"
DB_PASSWORD = "postgres"
DB_HOST = "localhost"

# Path on Google Drive to the backup file
BACKUP_PATH = DEV_PATH + "/synthea_ehr_backup.sql"

def restore_database():
    """Restores the synthea_ehr database from a file on Google Drive."""
    try:
        print("Starting database restore...")

        # First, drop and re-create the database to ensure a clean state
        print("Dropping and re-creating the database for a clean restore...")
        env = os.environ.copy()
        env['PGPASSWORD'] = DB_PASSWORD

        # Command to drop the database
        drop_command = [
            'dropdb',
            '--host', DB_HOST,
            '--username', DB_USER,
            DB_NAME
        ]
        # This will fail if the DB doesn't exist, so we don't check for errors
        subprocess.run(drop_command, env=env, check=False, capture_output=True, text=True)

        # Command to create the database
        create_command = [
            'createdb',
            '--host', DB_HOST,
            '--username', DB_USER,
            DB_NAME
        ]
        subprocess.run(create_command, env=env, check=True, capture_output=True, text=True)
        print("Database re-created successfully.")

        # Use subprocess to run the psql command to restore the backup
        command = [
            'psql',
            '--host', DB_HOST,
            '--username', DB_USER,
            '--dbname', DB_NAME,
            '--file', BACKUP_PATH
        ]

        process = subprocess.run(command, env=env, check=True, capture_output=True, text=True)
        print("Database restore successful!")

    except FileNotFoundError:
        print("Error: psql or dropdb/createdb commands not found. Please ensure PostgreSQL client tools are installed.")
        print("You can try running: !apt-get update && !apt-get install -y postgresql-client")
    except subprocess.CalledProcessError as e:
        print("Error during restore process.")
        print(f"Subprocess returned non-zero exit code: {e.returncode}")
        print(f"STDOUT:\n{e.stdout}")
        print(f"STDERR:\n{e.stderr}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

restore_database()

Starting database restore...
Dropping and re-creating the database for a clean restore...
Database re-created successfully.
Database restore successful!


In [10]:
# If needed once per runtime:
# !pip -q install psycopg2-binary

import re
import psycopg2
import psycopg2.extras

# Uses your existing globals:
# DB_NAME = "synthea_ehr"
# DB_USER = "postgres"
# DB_PASSWORD = "postgres"
# DB_HOST = "localhost"

# --- helpers ---
_SELECT_ONLY = re.compile(r"^\s*select\b", re.IGNORECASE | re.DOTALL)
_BANNED = (" insert ", " update ", " delete ", " drop ", " alter ",
           " create ", " grant ", " revoke ", " truncate ", " copy ", " do ")

# match :name (not preceded by another :)
_PARAM = re.compile(r'(?<!:):([a-zA-Z_]\w*)')

def _clean_sql(text: str) -> str:
    """Remove code fences / labels and invisible chars."""
    s = (text or "").strip()
    if s.lower().startswith("sql:"):
        s = s[4:].strip()
    m = re.match(r"^```(?:\s*sql)?\s*([\s\S]*?)\s*```$", s, flags=re.IGNORECASE)
    if m:
        s = m.group(1).strip()
    return s.replace("\ufeff","").replace("\u200b","").strip()

def _is_safe_select(sql: str) -> bool:
    """Single SELECT only; no DDL/DML keywords; no mid-string semicolons."""
    s = sql.strip()
    if ";" in s[:-1]:  # allow a single trailing semicolon only
        return False
    if not _SELECT_ONLY.match(s):
        return False
    low = f" {s.lower()} "
    return not any(b in low for b in _BANNED)

def _to_psycopg2_named(sql: str) -> str:
    """Convert :name placeholders to %(name)s for psycopg2."""
    return _PARAM.sub(r"%(\1)s", sql)

# --- main function ---
def execute_sql(sql: str, params: dict = None, timeout_ms: int = 5000):
    """
    Execute a single SELECT query safely and return rows as a list of dicts.

    Args:
      sql: SQL string (can use :param style placeholders, e.g., :patient_id)
      params: dict of parameters if placeholders are used (optional)
      timeout_ms: statement timeout in milliseconds (default 5000)

    Returns:
      List[Dict]: each row as a dict
    """
    raw = _clean_sql(sql)
    if not _is_safe_select(raw):
        raise ValueError("Blocked: SQL must be a single SELECT without DDL/DML.")

    q = _to_psycopg2_named(raw)
    params = params or {}

    conn = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=5432,
        connect_timeout=5,
        options=f"-c statement_timeout={timeout_ms}"
    )
    try:
        with conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor) as cur:
            cur.execute(q, params)
            return [dict(r) for r in cur.fetchall()]
    finally:
        conn.close()


In [11]:
# Test

# Simple
rows = execute_sql("SELECT COUNT(*) AS n FROM patients;")
print(rows)


[{'n': 111}]


In [12]:
rows = execute_sql("select patient_id from patients;")
print(rows[:3])

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'}, {'patient_id': '782001bc-f712-50ae-04f5-9a488f3ef4aa'}, {'patient_id': '80e7f50a-3e99-d5ac-cf97-f8a4b4f9e6c7'}]


In [13]:
execute_sql("""
SELECT column_name
FROM information_schema.columns
WHERE table_schema='public' AND table_name='encounters'
ORDER BY 1;
""")

[{'column_name': 'class'},
 {'column_name': 'encounter_id'},
 {'column_name': 'end_datetime'},
 {'column_name': 'patient_id'},
 {'column_name': 'reason_text'},
 {'column_name': 'start_datetime'}]

In [14]:
execute_sql("""
SELECT table_name, column_name
FROM information_schema.columns
WHERE table_schema='public'
ORDER BY 1;
""")

[{'table_name': 'conditions', 'column_name': 'onset_datetime'},
 {'table_name': 'conditions', 'column_name': 'patient_id'},
 {'table_name': 'conditions', 'column_name': 'encounter_id'},
 {'table_name': 'conditions', 'column_name': 'code'},
 {'table_name': 'conditions', 'column_name': 'display'},
 {'table_name': 'conditions', 'column_name': 'condition_id'},
 {'table_name': 'conditions', 'column_name': 'abatement_datetime'},
 {'table_name': 'encounters', 'column_name': 'reason_text'},
 {'table_name': 'encounters', 'column_name': 'start_datetime'},
 {'table_name': 'encounters', 'column_name': 'end_datetime'},
 {'table_name': 'encounters', 'column_name': 'encounter_id'},
 {'table_name': 'encounters', 'column_name': 'patient_id'},
 {'table_name': 'encounters', 'column_name': 'class'},
 {'table_name': 'immunizations', 'column_name': 'base_cost'},
 {'table_name': 'immunizations', 'column_name': 'code'},
 {'table_name': 'immunizations', 'column_name': 'date'},
 {'table_name': 'immunizations', 

In [15]:
# With parameter (your canonical pattern)
q = """
SELECT DISTINCT ON (mr.patient_id, mr.med_name)
  mr.patient_id,
  mr.med_name AS medication,
  mr.dose,
  mr.route,
  mr.start_datetime,
  mr.end_datetime,
  mr.refills
FROM medication_requests mr
WHERE mr.patient_id = :patient_id
  AND (mr.end_datetime IS NULL OR mr.end_datetime >= NOW())
ORDER BY mr.patient_id,
         mr.med_name,
         COALESCE(mr.end_datetime, mr.start_datetime) DESC NULLS LAST;

"""
rows = execute_sql(q, {"patient_id": '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'})
print(rows[:3])

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40', 'medication': 'Acetaminophen 325 MG Oral Tablet', 'dose': None, 'route': None, 'start_datetime': None, 'end_datetime': None, 'refills': None}, {'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40', 'medication': 'Naproxen sodium 220 MG Oral Tablet', 'dose': None, 'route': None, 'start_datetime': None, 'end_datetime': None, 'refills': None}]


# Tie LLM output to return results from PostGre SQL database

In [16]:
def answer_patient_question(user_question: str, patient_id: str, k: int = 3, max_tokens: int = 400 ):
    sql = generate_sql(user_question, k=k, max_tokens=max_tokens)
    #print("answer_patient_question: Generated SQL:\n", sql, "\n")
    rows = execute_sql(sql, {"patient_id": patient_id})
    #print("answer_patient_question: Executed SQL returned rows:\n", len(rows), "\n")
    return sql, rows


In [17]:
execute_sql("select p.patient_id from patients p")

[{'patient_id': '8c8e1c9a-b310-43c6-33a7-ad11bad21c40'},
 {'patient_id': '782001bc-f712-50ae-04f5-9a488f3ef4aa'},
 {'patient_id': '80e7f50a-3e99-d5ac-cf97-f8a4b4f9e6c7'},
 {'patient_id': 'edc17058-55fb-08c7-12df-ece93a402e50'},
 {'patient_id': '9f9dbdcb-23a1-82cc-b7bc-e0e420a95bd1'},
 {'patient_id': 'be874504-c868-ebfd-9a77-df6b1e5ff6cc'},
 {'patient_id': '30e48e16-2df7-207e-7a3d-1650ef0c1ed8'},
 {'patient_id': '57b21dea-ff00-6c3e-92d9-91c7627f53b2'},
 {'patient_id': 'a3d34c1f-5421-e078-38ec-1498a5941dbe'},
 {'patient_id': 'e83fe1b3-f94f-5591-f851-1da300e24e99'},
 {'patient_id': 'e6705c33-7349-8b12-484d-3b1f93227178'},
 {'patient_id': '2da86d63-34ae-b887-ddff-8f6f1e6990f1'},
 {'patient_id': '04181caa-fcc1-c6c8-743e-a903eff368de'},
 {'patient_id': '20802592-1c31-7339-4c4c-2fe648e1a716'},
 {'patient_id': '406e8bad-81b5-7624-5b8a-4aeeb74028f5'},
 {'patient_id': 'a331b5bc-cbea-a205-a8bf-dbf3255ef36a'},
 {'patient_id': '641efcda-7397-4172-c6ac-8231342fa53e'},
 {'patient_id': 'e64918a6-528c-

In [18]:
sql = (
    "SELECT e.patient_id, e.start_datetime, e.end_datetime, "
    "e.class AS encounter_class, e.reason_text AS reason "
    "FROM encounters e "
    "WHERE e.reason_text IS NOT NULL;"
)
execute_sql(sql)

[]

In [19]:
import pandas as pd

patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = "List all my vital signs"

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
print("Generated SQL:\n", sql, "\n")
print("Rows:", len(rows))
if rows:
    display(pd.DataFrame(rows).head(10))

Generated SQL:
 SELECT
  o.patient_id,
  COALESCE(o.display, o.loinc_code) AS vital_name,
  o.value_num AS value,
  o.value_unit AS unit,
  o.effective_datetime
FROM observations AS o
WHERE o.patient_id = :patient_id
  AND (
    o.loinc_code IN ('8480-6','8462-4','8867-4','9279-1','8310-5','59408-5','29463-7','39156-5','8302-2')
    OR LOWER(o.display) IN (
      'systolic blood pressure','diastolic blood pressure',
      'heart rate','respiratory rate','body temperature',
      'oxygen saturation','body weight','bmi','body mass index','body height'
    )
  )
ORDER BY
  o.effective_datetime DESC NULLS LAST,
  COALESCE(o.display, o.loinc_code); 

Rows: 56


,patient_id,vital_name,value,unit,effective_datetime
0,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Height,183.4,cm,2025-04-21 14:19:34+00:00
1,0fca905f-391c-08d3-4b93-b53f69b9da53,Body mass index (BMI) [Ratio],28.35,kg/m2,2025-04-21 14:19:34+00:00
2,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Weight,95.4,kg,2025-04-21 14:19:34+00:00
3,0fca905f-391c-08d3-4b93-b53f69b9da53,Heart rate,69,/min,2025-04-21 14:19:34+00:00
4,0fca905f-391c-08d3-4b93-b53f69b9da53,Respiratory rate,14,/min,2025-04-21 14:19:34+00:00
5,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Height,183.4,cm,2024-04-15 14:19:34+00:00
6,0fca905f-391c-08d3-4b93-b53f69b9da53,Body mass index (BMI) [Ratio],28.26,kg/m2,2024-04-15 14:19:34+00:00
7,0fca905f-391c-08d3-4b93-b53f69b9da53,Body Weight,95.1,kg,2024-04-15 14:19:34+00:00
8,0fca905f-391c-08d3-4b93-b53f69b9da53,Heart rate,83,/min,2024-04-15 14:19:34+00:00
9,0fca905f-391c-08d3-4b93-b53f69b9da53,Respiratory rate,15,/min,2024-04-15 14:19:34+00:00


# Post data frame, LLM genberates a nice summary

In [20]:
import pandas as pd
from openai import OpenAI
import io

client = OpenAI()  # assumes OPENAI_API_KEY is set


In [21]:
def df_to_csv_for_llm(df: pd.DataFrame, max_rows: int = 200, null_marker: str = "—") -> tuple[str, bool]:
    """
    Convert a DataFrame to CSV for the LLM.
    - Truncates to max_rows to keep prompts small.
    - Replaces NaNs with a visible marker (default "—").
    Returns (csv_text, truncated_flag).
    """
    truncated = False
    if len(df) > max_rows:
        df = df.head(max_rows).copy()
        truncated = True

    df = df.copy()
    df = df.fillna(null_marker)

    # keep column order stable
    csv_buf = io.StringIO()
    df.to_csv(csv_buf, index=False)
    return csv_buf.getvalue(), truncated

def summarize_df_with_llm(
    df: pd.DataFrame,
    patient_id: str,
    model: str = "gpt-4o-mini",
    max_rows: int = 200,
    null_marker: str = "—",
    max_tokens: int = 500
) -> str:
    """
    Ask the LLM to summarize a DataFrame.
    - Includes all visible (non-missing) values in its reasoning context,
      but the model will produce a concise natural-language summary (not a reprint of all rows).
    """
    if df is None or df.empty:
        return "No data found for the requested query."

    csv_text, truncated = df_to_csv_for_llm(df, max_rows=max_rows, null_marker=null_marker)
    columns_csv = ",".join(list(df.columns))

    user_prompt = f"""
You are a precise medical data summarizer. Use only the table below.
- Do not invent values or fields.
- Call out trends, counts, notable recency, and any obvious gaps (fields marked "{null_marker}").
- Keep it concise (2–5 sentences).
- If the table was truncated, say so and include how many rows were shown.

Patient: {patient_id}
Columns: {columns_csv}
Rows shown: {min(len(df), max_rows)}{' (truncated)' if truncated else ''}

CSV:
{csv_text}
""".strip()

    resp = client.chat.completions.create(
        model=model,
        temperature=0,
        messages=[
            {"role": "system", "content": "You are a precise, conservative medical data summarizer."},
            {"role": "user", "content": user_prompt},
        ],
        max_tokens=max_tokens,
    )
    return resp.choices[0].message.content.strip()


In [22]:
# ##
# patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
# user_q = "What has been my highest weight"

# sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
# print("Generated SQL:\n", sql, "\n")
# print("Rows:", len(rows))
# # if rows:
# #     display(pd.DataFrame(rows).head(10))
# df = pd.DataFrame(rows)
# display(df.head(10))
# # 3) Summarize with the LLM
# summary = summarize_df_with_llm(df, patient_id="<REAL-PATIENT-ID>", model=LLM_MODEL)
# print(summary)

# Gradio UI

In [23]:
# 0) Install Gradio (quiet)
!pip -q install gradio

# 1) Build a small adapter for the UI
import pandas as pd
import gradio as gr

def medintellagent_ui(patient_id: str, user_question: str, k: int = 3, max_tokens: int = 600):
    if not patient_id or not user_question:
        return "Please provide both patient_id and a question.", pd.DataFrame(), "—"
    try:
        # Generate SQL + fetch rows
        sql, rows = answer_patient_question(user_question, patient_id, k=k, max_tokens=max_tokens)
        df = pd.DataFrame(rows)
        if df.empty:
            summary = "No data found for this query (for this patient). Try another question or patient_id."
        else:
            summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
        return sql, df, summary
    except Exception as e:
        # Show errors cleanly in the UI (helpful during dev)
        return f"Error: {e}", pd.DataFrame(), "—"

# 2) Build the interface
with gr.Blocks(css="footer {visibility: hidden}") as demo:
    gr.Markdown("# MedIntellAgent — MVP UI")
    gr.Markdown("Enter a **Patient ID** and **Question**. You’ll get the generated SQL, raw DB rows, and an LLM summary.")

    with gr.Row():
        patient = gr.Textbox(label="Patient ID", placeholder="e.g., 0fca905f-391c-08d3-4b93-b53f69b9da53")
    question = gr.Textbox(label="Patient Question", lines=2,
                          placeholder="e.g., Which medications am I currently taking?")
    with gr.Row():
        k = gr.Slider(1, 5, value=3, step=1, label="Few-shot k")
        max_tokens = gr.Slider(100, 2000, value=600, step=50, label="LLM max_tokens")

    ask = gr.Button("Ask MedIntellAgent")

    sql_out = gr.Code(label="Generated SQL", language="sql")
    table_out = gr.Dataframe(label="Postgres Results", interactive=False)
    summary_out = gr.Markdown(label="LLM Summary")

    ask.click(medintellagent_ui,
              inputs=[patient, question, k, max_tokens],
              outputs=[sql_out, table_out, summary_out])

demo.launch(share=True)   # share=True if you want a temporary public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bee96f7a1bdc08189d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# Evaluation



1.   For now we will use Open AI as LLM for generating Optimized response summary, and score evaluation. Down the line we may change this to e.g. Calude




In [24]:
%%capture
pip install -U langgraph

In [25]:
%%capture
pip install -U langchain_core langchain_openai

In [26]:
from pydantic import BaseModel, Field

In [27]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [28]:
from typing import TypedDict
from typing import Literal
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate


def generate_empathy_score(user_query: str, response: str):

    #Graph State
    class State(TypedDict):
      user_query: str
      response: str
      empathy_score: str
      is_empathetic: bool


    #Schema for structured outout to use in evaluation
    class Feedback(BaseModel):
      score: Literal ["1","2","3","4","5"] = Field(
          description="Provide a score for how empathetic is the response. 5 is highest empathy."
      )
      feedback: str = Field(
          description="Provide a score for how empathetic is the response. Score of 1: response has no empathy. Score of 5: response has excellent empathy for a human."

      )

    evaluator = llm.with_structured_output(Feedback)

    seeded_data = {
    "user_query": user_query,
    "response": response
    }
    # --- 3. CREATE PROMPT TEMPLATE ---

    # Define the evaluation instructions using a system message and placeholders
    prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """You are an expert, highly empathetic evaluator.
                Your task is to judge the system generated response to a user (patient) query.
                You must provide a score (1-5) and detailed feedback on the empathy level of the response, strictly following the provided JSON schema.

                --- Context ---
                User Query: {user_query}
                Response to Evaluate: {response}
                """
            ),
            ("human", "Evaluate the response for empathy and provide the JSON output.")
        ]
    )

    # Prepare the data for the prompt
    # We convert the table to a string for the prompt
    prompt_data = {
        "user_query": seeded_data["user_query"],
        "response": seeded_data["response"]
    }

    # --- 4. RUN EVALUATION ---

    # Create the runnable chain: Prompt -> LLM with Structured Output
    evaluation_chain = prompt_template | evaluator

    # Invoke the chain
    try:
        evaluation_result: Feedback = evaluation_chain.invoke(prompt_data)

        # Output the result
        print("--- EVALUATION RESULT ---")
        print(f"Empathy Score: {evaluation_result.score}/5")
        print(f"Feedback: {evaluation_result.feedback}")

        return {"empathy_score": evaluation_result.score, "feedback": evaluation_result.feedback}
    except Exception as e:
        print(f"An error occurred during evaluation: {e}")




In [29]:
#Testing of empathy score

patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'which medication am I taking?'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)


In [30]:
summary

'The table shows medication data for a single patient (ID: 0fca905f-391c-08d3-4b93-b53f69b9da53) with five entries. All medications listed, including Acetaminophen, oxyCODONE, and amLODIPine, have unspecified start and end dates, as well as no refill information. Notably, there is a gap in the dosage for Naproxen sodium. The data appears truncated, as only five rows are shown.'

In [31]:
generate_empathy_score(user_q,summary)

--- EVALUATION RESULT ---
Empathy Score: 1/5
Feedback: The response lacks any empathetic language or acknowledgment of the user's situation. It is purely factual and does not address the user's need for clarity or reassurance regarding their medication. There is no attempt to connect with the user on an emotional level or to provide support, making it feel cold and impersonal.


{'empathy_score': '1',
 'feedback': "The response lacks any empathetic language or acknowledgment of the user's situation. It is purely factual and does not address the user's need for clarity or reassurance regarding their medication. There is no attempt to connect with the user on an emotional level or to provide support, making it feel cold and impersonal."}

In [32]:
user_q = 'List all my vital signs'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [33]:
summary

'The data for patient 0fca905f-391c-08d3-4b93-b53f69b9da53 includes 56 rows of vital measurements, with the most recent entries dated April 21, 2025. Notable trends include a decrease in Body Weight from 102 kg in April 2023 to 95.4 kg in April 2025, and a corresponding decrease in Body Mass Index (BMI) from 30.33 kg/m² to 28.35 kg/m². Heart rate values show variability, with a high of 99 beats per minute in March 2017 and a recent low of 69 beats per minute in April 2025. There are gaps in the data for Body Temperature, which is only recorded once in June 2017. The table appears to be truncated, showing 56 rows.'

In [34]:
generate_empathy_score(user_q,summary)

--- EVALUATION RESULT ---
Empathy Score: 2/5
Feedback: The response provides factual information about the user's vital signs but lacks any empathetic language or acknowledgment of the user's feelings or concerns. It presents data in a clinical manner without addressing the user's potential anxiety or curiosity about their health. A more empathetic response would include reassurance, an invitation for the user to ask questions, or a recognition of the importance of these measurements to the user's overall well-being.


{'empathy_score': '2',
 'feedback': "The response provides factual information about the user's vital signs but lacks any empathetic language or acknowledgment of the user's feelings or concerns. It presents data in a clinical manner without addressing the user's potential anxiety or curiosity about their health. A more empathetic response would include reassurance, an invitation for the user to ask questions, or a recognition of the importance of these measurements to the user's overall well-being."}

In [35]:
user_q = 'what has been my weight history'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [36]:
summary

'The data shows a trend of fluctuating body weight for the patient over several years, with the most recent measurement on April 21, 2025, at 95.4 kg. Notably, there was a peak weight of 102 kg recorded in both April 2023 and March 2018. The values indicate a general decrease in weight from 102 kg in 2023 to 95.4 kg in 2025, suggesting a potential weight loss trend. There are no entries for vital signs other than body weight, and the table is truncated, showing only 11 rows.'

In [37]:
score_response = generate_empathy_score(user_q,summary)

--- EVALUATION RESULT ---
Empathy Score: 2/5
Feedback: The response provides factual information about the user's weight history, including specific measurements and trends. However, it lacks empathy and emotional support. It does not acknowledge the user's feelings or concerns regarding their weight history, nor does it offer encouragement or understanding. A more empathetic response would include validation of the user's experience and perhaps a supportive comment about their weight loss trend.


In [38]:
print(score_response)

{'empathy_score': '2', 'feedback': "The response provides factual information about the user's weight history, including specific measurements and trends. However, it lacks empathy and emotional support. It does not acknowledge the user's feelings or concerns regarding their weight history, nor does it offer encouragement or understanding. A more empathetic response would include validation of the user's experience and perhaps a supportive comment about their weight loss trend."}


# Now after these trial runs we aspire to optimize the empathy score

In [39]:
from typing import TypedDict, Literal, Optional
import pandas as pd
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage # NEW IMPORT
from langgraph.graph import StateGraph, START, END

# --- 1. CONFIGURATION AND SCHEMA ---

# Note: This requires the OPENAI_API_KEY environment variable to be set.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 1.1 Graph State (TypedDict)
class PatientState(TypedDict):
    """The state of the conversation and evaluation."""
    patient_query: str
    patient_data: str # Using str to hold Markdown representation of DataFrame
    response: str
    empathy_score: int
    evaluation_feedback: str
    retry_count: int

# 1.2 Structured Output Schema (Pydantic)
class EmpathyEvaluation(BaseModel):
    """Schema for the structured output of the evaluation LLM."""
    score: Literal[1, 2, 3, 4, 5] = Field(
        description="The empathy score for the response (1=low empathy, 5=high empathy)."
    )
    feedback: str = Field(
        description="Constructive, actionable feedback on how to improve the response's empathy, especially if the score is 3 or less."
    )

# Augment the LLM for structured evaluation output
evaluator_llm = llm.with_structured_output(EmpathyEvaluation)


# --- 2. GRAPH NODES ---

def generate_response(state: PatientState) -> dict:
    """
    Node 1: Generates or regenerates a patient response using the LLM.
    """
    print(f"\n--- GENERATING RESPONSE (Retry: {state['retry_count']}) ---")

    # Base instructions content
    system_prompt_content = (
        "You are a highly empathetic and professional clinical assistant. "
        "Your primary goal is to acknowledge the patient's feelings and provide clear, gentle, and helpful next steps. "
        "The current patient query and their data are provided below. Respond with empathy first."
    )

    # Context message content (always included)
    context_message_content = (
        f"Patient Query: {state['patient_query']}\nPatient Data Snapshot:\n{state['patient_data']}"
    )

    # Determine user prompt content
    if state["retry_count"] > 0:
        # Retry prompt: incorporate previous feedback
        user_prompt_content = (
            f"RETRY RESPONSE. The previous attempt scored {state['empathy_score']}/5. "
            f"The feedback was: '{state['evaluation_feedback']}'. "
            f"Write a new response that explicitly addresses this feedback and improves empathy."
        )
    else:
        # Initial prompt
        user_prompt_content = "Generate a response."

    # Construct the final list of BaseMessages (REQUIRED INPUT TYPE)
    messages = [
        SystemMessage(content=system_prompt_content),
        HumanMessage(content=context_message_content),
        HumanMessage(content=user_prompt_content)
    ]

    # Invoke LLM with the list of messages
    msg = llm.invoke(messages)

    return {
        "response": msg.content,
        "retry_count": state["retry_count"] + 1,
        "evaluation_feedback": "" # Clear feedback before next evaluation
    }


def evaluate_response(state: PatientState) -> dict:
    """
    Node 2: Evaluates the generated response for empathy using the structured LLM.
    """
    print("\n--- EVALUATING RESPONSE ---")

    # Prompt for the structured evaluator
    evaluation_prompt = (
        f"Evaluate the following clinical assistant response for empathy on a scale of 1 to 5. "
        f"Score 1 means cold/robotic. Score 5 means outstanding empathy and warmth. "
        f"Patient Query: {state['patient_query']}\n"
        f"Assistant Response: {state['response']}"
    )

    # Invoke the structured LLM
    evaluation_result: EmpathyEvaluation = evaluator_llm.invoke(evaluation_prompt)

    print(f"-> Score: {evaluation_result.score}/5")

    return {
        "empathy_score": evaluation_result.score,
        "evaluation_feedback": evaluation_result.feedback
    }


def route_to_retry(state: PatientState) -> str:
    """
    Conditional Edge: Decides whether to loop back for regeneration or end.
    """
    score = state["empathy_score"]
    max_retries = 3 # Safety limit to prevent infinite loops

    if score > 3:
        print("\n*** ROUTE: END (Score > 3) ***")
        return "end"
    elif state["retry_count"] >= max_retries:
        print(f"\n*** ROUTE: END (Max Retries ({max_retries}) Reached) ***")
        return "end"
    else:
        print("\n*** ROUTE: RETRY (Score <= 3) ***")
        return "retry"


# --- 3. WORKFLOW ASSEMBLY AND MAIN FUNCTION ---

def optimize_patient_response(query: str, data: pd.DataFrame, max_retries=3):
    """
    Main function to run the optimization graph.

    Args:
        query (str): The patient's input question or concern.
        data (pd.DataFrame): Relevant patient data.
        max_retries (int): Maximum number of attempts before stopping.

    Returns:
        dict: The final state of the graph, including the optimized response.
    """

    # Initialize the graph builder
    optimizer_builder = StateGraph(PatientState)

    # Add the nodes (actions)
    optimizer_builder.add_node("generator", generate_response)
    optimizer_builder.add_node("evaluator", evaluate_response)

    # Define the entry point
    optimizer_builder.add_edge(START, "generator")

    # Define the flow from generation to evaluation
    optimizer_builder.add_edge("generator", "evaluator")

    # Define the conditional loop after evaluation
    optimizer_builder.add_conditional_edges(
        "evaluator",
        route_to_retry,
        {
            "retry": "generator", # Loop back to generator
            "end": END,           # Stop the workflow
        }
    )

    # Compile the workflow
    optimizer_workflow = optimizer_builder.compile()

    # Initial state preparation
    initial_state = {
        "patient_query": query,
        "patient_data": data.to_markdown(), # Convert DataFrame head to readable string
        "response": "",
        "empathy_score": 0,
        "evaluation_feedback": "",
        "retry_count": 0
    }

    print(f"--- STARTING OPTIMIZATION (Max Retries: {max_retries}) ---")

    # Invoke the workflow
    final_state = optimizer_workflow.invoke(initial_state, {"recursion_limit": max_retries + 2})

    return final_state

In [40]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'which medication am I taking?'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)


In [41]:
summary_1 = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [42]:
summary_1

'The table shows medication data for a single patient (ID: 0fca905f-391c-08d3-4b93-b53f69b9da53) with five entries. All medications listed, including Acetaminophen, oxyCODONE, and amLODIPine, have unspecified start and end dates, as well as no refill information. Notably, one entry for Naproxen sodium lacks dosage details. The data appears truncated, as only five rows are shown, and there are gaps in critical fields.'

In [43]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: which medication am I taking?
Final Score: 4/5
Final Response:
I understand that it can be a bit confusing to keep track of your medications, and I'm here to help you with that. Based on your records, you are currently taking the following medications:

1. **Acetaminophen 325 MG Oral Tablet (Tylenol)**
2. **Acetaminophen 325 MG / OxyCODONE Hydrochloride 5 MG Oral Tablet**
3. **Amlodipine 2.5 MG Oral Tablet**
4. **Naproxen Sodium 220 MG Oral Tablet**

If you have any questions about these medications, such as their purposes or how to take them, please feel free to ask. It's important to feel comfortable and informed about your treatment.


**Test 2**




In [44]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'what has been my weight history'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [45]:
summary

'The data for patient 0fca905f-391c-08d3-4b93-b53f69b9da53 shows a trend of fluctuating body weight over the years, with the most recent measurement on April 21, 2025, at 95.4 kg. Notably, there was a peak weight of 102 kg recorded in both April 2023 and March 2018. The values indicate a general decrease in weight from 102 kg in 2023 to 95.4 kg in 2025, suggesting a potential weight loss trend. There are no entries for vital signs other than body weight, indicating a gap in the data. The table shows 11 rows of data.'

In [46]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: what has been my weight history
Final Score: 4/5
Final Response:
I understand that tracking your weight history can be important for your health and well-being, and I appreciate you reaching out about this. Here’s a summary of your weight history over the past several years:

- **2025-04-21**: 95.4 kg
- **2024-04-15**: 95.1 kg
- **2023-04-10**: 102 kg
- **2022-04-04**: 99.6 kg
- **2021-03-29**: 97.3 kg
- **2020-11-16**: 96.4 kg
- **2020-03-23**: 94.9 kg
- **2019-03-18**: 95.2 kg
- **2018-03-12**: 102 kg
- **2017-03-06**: 100 kg
- **2016-02-29**: 98.1 kg

It looks like your weight has fluctuated over the years, with some notable changes. If you have any specific concerns or goals regarding your weight, or if you would like to discuss this further, please let me know. I'm here to 

**Test 3**

In [47]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'List all my vital signs'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [48]:
summary

'The data for patient 0fca905f-391c-08d3-4b93-b53f69b9da53 includes 56 rows of vital measurements, with the most recent entries dated April 21, 2025. Notable trends include a decrease in Body Weight from 102 kg in April 2023 to 95.4 kg in April 2025, and a corresponding decrease in Body Mass Index (BMI) from 30.33 kg/m² to 28.35 kg/m². Heart rate has fluctuated, with a notable drop from 88 beats per minute in April 2023 to 69 in April 2025. There are no entries for body temperature in the recent data, indicating a potential gap in monitoring this vital sign. The table appears to be truncated, showing only 56 rows.'

In [49]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: List all my vital signs
Final Score: 4/5
Final Response:
I understand that you’re looking for a comprehensive overview of your vital signs, and I’m here to help you with that. It’s important to keep track of these measurements as they can provide valuable insights into your health.

Here’s a summary of your vital signs:

1. **Body Height**: 
   - 183.4 cm (most recent measurement on April 21, 2025)

2. **Body Weight**: 
   - 95.4 kg (most recent measurement on April 21, 2025)

3. **Body Mass Index (BMI)**: 
   - 28.35 kg/m² (most recent measurement on April 21, 2025)

4. **Heart Rate**: 
   - 69 beats per minute (most recent measurement on April 21, 2025)

5. **Respiratory Rate**: 
   - 14 breaths per minute (most recent measurement on April 21, 2025)

If you have any specific c

**Test 4**

In [50]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'Show my BMI trend'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

In [51]:
summary

'The data shows a trend of decreasing Body Mass Index (BMI) values over the last several years, with the most recent measurement on April 21, 2025, at 28.35 kg/m², down from a peak of 30.33 kg/m² on April 10, 2023. Notably, the BMI has decreased from 30.32 kg/m² in 2018 to 28.35 kg/m² in 2025. There are no gaps in the vital measurements, but the data is truncated, showing only 11 rows.'

In [52]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: Show my BMI trend
Final Score: 4/5
Final Response:
I understand that tracking your BMI trend can be important for your health and wellness journey. It’s great that you’re taking the time to monitor these changes. 

Here’s a summary of your BMI trend over the past few years:

- **2025-04-21**: 28.35 kg/m²
- **2024-04-15**: 28.26 kg/m²
- **2023-04-10**: 30.33 kg/m²
- **2022-04-04**: 29.62 kg/m²
- **2021-03-29**: 28.91 kg/m²
- **2020-11-16**: 28.66 kg/m²
- **2020-03-23**: 28.21 kg/m²
- **2019-03-18**: 28.31 kg/m²
- **2018-03-12**: 30.32 kg/m²
- **2017-03-06**: 29.74 kg/m²
- **2016-02-29**: 29.16 kg/m²

From this data, we can see that your BMI has shown some fluctuations over the years, with a noticeable decrease from 30.33 kg/m² in 2023 to 28.35 kg/m² in 2025. This trend may indica

**Test 5**

In [53]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'Give me my blood pressure readings over time'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 No data found for the requested query.


In [54]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: Give me my blood pressure readings over time
Final Score: 4/5
Final Response:
I understand that keeping track of your blood pressure readings is important for your health, and it can be concerning if you’re unsure about your numbers over time. Unfortunately, I don’t have access to your specific blood pressure readings at the moment. 

However, I recommend checking with your healthcare provider or the facility where you have your readings recorded. They should be able to provide you with a detailed history of your blood pressure measurements. 

If you have been keeping a personal log or using a home monitor, reviewing that data can also be very helpful. If you have any questions about what your readings mean or how to manage your blood pressure, please feel free to ask. I'm here 

**Test 6**

In [55]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'What conditions have I been diagnosed with?'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 The table shows 21 rows of patient data, with a mix of resolved and active conditions. Notably, there are 10 active findings, including full-time and part-time employment statuses, chronic low back pain, and social isolation, which may indicate ongoing challenges. Resolved conditions include various dental and viral disorders, with the most recent resolution occurring in 2024. There are several gaps in the data, particularly for the active conditions, which lack resolution dates.


In [56]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: What conditions have I been diagnosed with?
Final Score: 4/5
Final Response:
I understand that it can be concerning to keep track of your health conditions, and I'm here to help you with that. Based on your records, here are the conditions you have been diagnosed with:

**Active Conditions:**
1. **Body Mass Index 30+ - Obesity**
2. **Chronic Low Back Pain**
3. **Essential Hypertension**
4. **Social Isolation**
5. **Limited Social Contact**
6. **Victim of Intimate Partner Abuse**
7. **Stress**
8. **Chronic Pain**
9. **Has a Criminal Record**

**Resolved Conditions:**
1. **Primary Dental Caries**
2. **Gingival Disease**
3. **Acute Viral Pharyngitis**
4. **Viral Sinusitis**
5. **Gingivitis**
6. **Traumatic Dislocation of Temporomandibular Joint**
7. **Medication Review Due**

If yo

**Test 7**

In [57]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'How screwed up am I? '

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 No data found for the requested query.


In [58]:
df

""


In [59]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 5/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: How screwed up am I? 
Final Score: 5/5
Final Response:
I want to start by acknowledging how you're feeling right now. It’s completely normal to have moments of self-doubt or to feel overwhelmed, and it’s brave of you to reach out. Remember, everyone goes through tough times, and it doesn’t define who you are or your potential for healing and growth.

If you’re comfortable, I’d like to understand more about what’s been on your mind or what specific concerns you have. This can help us identify some next steps together. You’re not alone in this, and there are ways to find support and clarity.


**Test 8**

In [60]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'LIST my active prescriptions'

sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
df = pd.DataFrame(rows)
summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
print("Non-optimized summary: \n",summary)

Non-optimized summary: 
 The table shows medication records for a single patient (ID: 0fca905f-391c-08d3-4b93-b53f69b9da53) with five entries. All medications listed, including Acetaminophen, oxyCODONE, and amLODIPine, have a dose of 1.0, but the route, start and end dates, and refills are not provided (marked as "—"). Notably, there is a gap in the data regarding the administration details and duration of treatment. The table is truncated, showing only 5 rows.


In [61]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 4/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: LIST my active prescriptions
Final Score: 4/5
Final Response:
I understand that keeping track of your medications can be overwhelming at times, and I'm here to help you with that. Here’s a list of your active prescriptions:

1. **Acetaminophen 325 MG Oral Tablet [Tylenol]**
   - Dose: 1 tablet
   - Route: Oral

2. **Acetaminophen 325 MG / OxyCODONE Hydrochloride 5 MG Oral Tablet**
   - Dose: 1 tablet
   - Route: Oral

3. **Amlodipine 2.5 MG Oral Tablet**
   - Dose: 1 tablet
   - Route: Oral

4. **Naproxen Sodium 220 MG Oral Tablet**
   - Dose: Not specified
   - Route: Oral

If you have any questions about these medications or need further assistance, please feel free to ask. Your health and comfort are important, and I'm here to support you.


**Test 9**

In [65]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'Which of my visits have lab results'

try:
    sql, rows = answer_patient_question(user_q, patient_id, k=5, max_tokens=1000)
    df = pd.DataFrame(rows)
    summary = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)
    print("Non-optimized summary: \n",summary)
except Exception as error:
    print(error)


Blocked: SQL must be a single SELECT without DDL/DML.


In [66]:

df

,patient_id,medication,dose,route,start_datetime,end_datetime,refills
0,0fca905f-391c-08d3-4b93-b53f69b9da53,Acetaminophen 325 MG Oral Tablet [Tylenol],1.0,None,None,None,None
1,0fca905f-391c-08d3-4b93-b53f69b9da53,Acetaminophen 325 MG / oxyCODONE Hydrochloride...,1.0,None,None,None,None
2,0fca905f-391c-08d3-4b93-b53f69b9da53,amLODIPine 2.5 MG Oral Tablet,1.0,None,None,None,None
3,0fca905f-391c-08d3-4b93-b53f69b9da53,Naproxen sodium 220 MG Oral Tablet,None,None,None,None,None
4,0fca905f-391c-08d3-4b93-b53f69b9da53,None,None,None,None,None,None


In [67]:
# Run the optimization process
optimized_result = optimize_patient_response(
  query=user_q,
  data=df,
  max_retries=3 # Will stop after 3 attempts, even if score < 4
)

print("\n==============================================")
print("      FINAL OPTIMIZED RESPONSE & RESULTS")
print("==============================================")
print(f"Query: {optimized_result['patient_query']}")
print(f"Final Score: {optimized_result['empathy_score']}/5")
print(f"Final Response:\n{optimized_result['response']}")


--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 5/5

*** ROUTE: END (Score > 3) ***

      FINAL OPTIMIZED RESPONSE & RESULTS
Query: Which of my visits have lab results
Final Score: 5/5
Final Response:
I understand that you're looking for information about your visits that include lab results, and I can see how important this information is for you. It’s completely normal to want to keep track of your health and any tests that have been done.

To assist you better, I recommend checking your patient portal if you have access to it, as it often contains detailed information about your visits and any associated lab results. If you’re unable to find the information there, please consider reaching out to your healthcare provider’s office directly. They can provide you with a comprehensive list of your visits and any lab results associated with them.

If you have any other questions or need further assistance, plea

# LJ empathy optimized response (LLM as a Judge scores optimized response)



*   Relevance of response to patient question



In [68]:
user_question = "When was my last cholesterol test?"
portal_response = "Your most recent cholesterol test was on May 2, 2024, showing normal results."

judge_prompt_R = f"""
You are an expert judge that evaluates the quality of a response provided by a patient question-answering LLM system to a patient’s query.
Your task is to provide a Relevance score (1–5) for how well the system’s response answers the patient’s question about their own medical history or records.

***Criterion Definition***
RELEVANCE: The degree to which the response directly and appropriately addresses the specific patient question about their medical record, history, medications, lab results, appointments, or related data.

***Score Definitions and Examples***

1: NOT RELEVANT
- Meaning: The response does not relate to the patient’s question.
- Example:
  Q: “What medications am I currently taking?”
  A: “You should eat a balanced diet.”
  → Not relevant.

2: SLIGHTLY RELEVANT
- Meaning: The response touches the general topic but not the patient’s record.
- Example:
  Q: “When was my last blood test?”
  A: “Blood tests are usually done annually.”
  → Only generic information.

3: MODERATELY RELEVANT
- Meaning: The response partially answers or hints at relevant data but misses key specifics.
- Example:
  Q: “What medications am I currently taking?”
  A: “You are prescribed metformin, but I’m not sure about your other medications.”
  → Partial relevance.

4: HIGHLY RELEVANT
- Meaning: The response clearly addresses the question using mostly correct patient-specific data, minor omissions acceptable.
- Example:
  Q: “When was my last lab test?”
  A: “Your most recent lab test was in March 2024 for blood glucose.”
  → Very good relevance.

5: PERFECTLY RELEVANT
- Meaning: The response is direct, complete, and fully focused on the patient’s question and personal data.
- Example:
  Q: “What medications am I currently taking?”
  A: “You are currently taking Metformin 500 mg twice daily and Lisinopril 10 mg once daily.”
  → Excellent relevance.

Now evaluate the following pair:

User Question: {user_question}
Portal Response: {portal_response}

***Output Format***
Provide your output strictly as a JSON object:
- "User Question": "{user_question}",
- "Portal Response": "{portal_response}",
- "Relevance Score": <integer between 1 and 5>

Do not include explanations, reasoning, or any text outside this JSON object.
"""


In [69]:
df_as_text = df.to_markdown(index=False)

# Build the full Faithfulness evaluation prompt
judge_prompt_F = f"""You are an expert medical data evaluator assessing the factual correctness of a patient-portal system's response.

Your task is to provide a Faithfulness score (1–5) indicating how accurately the response reflects the factual information in the retrieved records.

***Criterion Definition***
FAITHFULNESS: The degree to which the response content is factually consistent with the evidence provided in the retrieved records below.
Do NOT judge relevance or helpfulness — only whether the statements are supported by the retrieved data.

***Score Definitions (with Domain-Relevant Examples)***

1 — NOT FAITHFUL
- Meaning: Mostly incorrect or hallucinated; contradicts the records.
- Example Evidence (Medications):
  | medication | dose  | freq |
  |------------|-------|------|
  | Metformin  | 500mg | BID  |
- Example Response: "You’re taking Metformin 500 mg BID and Atorvastatin 20 mg QD."
- Why: Atorvastatin is not in evidence → contradiction.

2 — SLIGHTLY FAITHFUL
- Meaning: Some correct facts, but significant unsupported claims remain.
- Example Evidence (Labs):
  | test | date       | result |
  |------|------------|--------|
  | A1C  | 2024-03-10 | 6.9%   |
- Example Response: "Your latest A1C on 2024-03-10 was 6.9%, and your LDL was 85 mg/dL."
- Why: A1C detail is correct; LDL is unsupported → major error persists.

3 — MODERATELY FAITHFUL
- Meaning: Mostly supported, but one or two factual inaccuracies (e.g., wrong dose/date).
- Example Evidence (Medications):
  | medication | dose | freq |
  |------------|------|------|
  | Lisinopril | 10mg | QD   |
- Example Response: "You take Lisinopril 5 mg once daily."
- Why: Correct med and frequency; dose is wrong → partial mismatch.

4 — HIGHLY FAITHFUL
- Meaning: Essentially correct; only minor discrepancy/omission (e.g., missing a less salient detail).
- Example Evidence (Appointments):
  | type             | date       | status    |
  |------------------|------------|-----------|
  | Cardio follow-up | 2025-06-14 | completed |
- Example Response: "Your last cardiology follow-up was in June 2025."
- Why: Month/year correct; missing exact day → minor omission.

5 — PERFECTLY FAITHFUL
- Meaning: Fully supported; no contradictions or unsupported claims.
- Example Evidence (Medications):
  | medication | dose  | freq |
  |------------|-------|------|
  | Metformin  | 500mg | BID  |
  | Lisinopril | 10mg  | QD   |
- Example Response: "You are taking Metformin 500 mg twice daily and Lisinopril 10 mg once daily."
- Why: Every stated fact matches the records exactly.

***Retrieved Records***
```markdown
{df_as_text}
Response to Evaluate
{portal_response}
Output Format
Return ONLY the following JSON object (no extra text):

{{
"User Question": "{user_question}",
"Portal Response": "{portal_response}",
"Faithfulness Score": <integer between 1 and 5>
}}

Judging Rules

Use ONLY the provided records for verification; do not infer or assume missing facts.
Formatting differences (e.g., "twice daily" vs "BID") are acceptable if semantically identical.
If no evidence supports a claimed fact, treat it as unsupported.
"""

In [70]:
# Build the Helpfulness evaluation prompt
judge_prompt_H = f"""
You are an expert medical communication evaluator. Judge how HELPFUL the patient-portal system's response is to the patient.

Your task is to provide a Helpfulness score (1–5) indicating whether the response meaningfully helps the patient act on or understand their own medical record question.

***Criterion Definition***
HELPFULNESS: The degree to which the response is immediately useful to the patient—i.e., it answers the question clearly and provides actionable guidance (when appropriate), next steps, or pointers to where the patient can find the information in their portal. Do NOT judge factual correctness (that's Faithfulness) or topicality (that's Relevance).

***Score Definitions (with Patient-Portal Examples)***

1 — NOT HELPFUL
- Meaning: Off-topic, evasive, or gives no usable guidance.
- Example Q: "When was my last cholesterol test?"
- Example A: "Cholesterol is a type of fat in the blood."
- Why: No answer, no steps.

2 — SLIGHTLY HELPFUL
- Meaning: Some topical info but still not actionable or patient-focused.
- Example Q: "What meds am I currently taking?"
- Example A: "Medications are often used to manage chronic conditions."
- Why: General info; provides no path to the patient’s own info.

3 — MODERATELY HELPFUL
- Meaning: Partially useful; gives a hint or partial answer but lacks clarity or next steps.
- Example Q: "Do I have any upcoming appointments?"
- Example A: "You can check upcoming visits in the Appointments section."
- Why: Direction is given, but no direct status or concrete steps (e.g., exact path).

4 — HIGHLY HELPFUL
- Meaning: Clear, patient-centered answer with minor omissions; includes concise steps or references.
- Example Q: "Where can I see my vaccine history?"
- Example A: "Open **Health Records → Immunizations** to view your vaccine history. If you don’t see recent entries, message your care team from **Messages → New Message**."
- Why: Concrete portal path and next step.

5 — PERFECTLY HELPFUL
- Meaning: Direct, concise answer tailored to the question, with clear steps and guardrails when appropriate.
- Example Q: "How can I refill my Metformin?"
- Example A: "Go to **Medications → Metformin → Refill**. If 'Refill' isn’t available, use **Messages → Pharmacy Request** or call the number on your prescription label. If you’re out of doses today, contact your pharmacy for an emergency supply."
- Why: Immediate action path, fallback, and safety guidance.

***Judging Rules***
- Focus ONLY on usefulness to the patient: clarity, directness, actionable steps, and appropriate guardrails.
- Prefer concise, structured guidance (e.g., portal navigation paths, who to contact, what to click).
- Do not penalize for not repeating data that belongs in the chart; penalize if it fails to guide the patient to it.
- Don’t reward unnecessary medical theory/jargon if it doesn’t help the patient do something.
- If the response can’t answer due to missing data, helpfulness increases when it explicitly states that and gives exact next steps to obtain it (e.g., where to look or whom to message).
- Safety: Encourage contacting clinicians for urgent symptoms; do not judge medical accuracy here.

***Input Pair to Evaluate***
User Question:
{user_question}

Portal Response:
{portal_response}

***Output Format***
Return ONLY the following JSON object (no extra text):

{{
  "User Question": "{user_question}",
  "Portal Response": "{portal_response}",
  "Helpfulness Score": <integer between 1 and 5>
}}
"""


# MedGemma-27B-text-it



*   MedGemma-27B-text-it instruction tuned better accuracy as a judge than Mistral 7B
*   QLoRA use on A100 40GB



In [71]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login # Import the login function

# Log in to Hugging Face
try:
    from google.colab import userdata
    hf_token = userdata.get('HF_TOKEN') # Assuming you have stored your HF token in Colab Secrets
    if hf_token:
        login(token=hf_token)
    else:
        print("Hugging Face token not found in Colab Secrets. Please add it as 'HF_TOKEN'.")
except Exception:
    print("Could not retrieve Hugging Face token from Colab Secrets.")
    # Fallback to manual input if needed, but not recommended for security
    # import getpass
    # hf_token = getpass.getpass("Enter your Hugging Face token: ")
    # login(token=hf_token)


model_id = "google/medgemma-27b-text-it"  # or the exact HF ID you use

bnb_cfg = BitsAndBytesConfig(load_in_4bit=True,
                             bnb_4bit_quant_type="nf4",
                             bnb_4bit_compute_dtype=torch.bfloat16,
                             bnb_4bit_use_double_quant=True)

tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_cfg,
    torch_dtype=torch.bfloat16
)



`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

In [72]:
def chat(prompt, max_new_tokens=256, temperature=0.0):
    inputs = tok(prompt, return_tensors="pt").to(model.device)

    # --- The necessary fix is here: determine do_sample based on temperature ---
    # When temperature is 0.0, you want greedy decoding (do_sample=False).
    # Otherwise, you want sampling (do_sample=True).
    do_sample_setting = temperature > 0.0

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            # Pass the corrected setting to the model.generate function
            do_sample=do_sample_setting
        )
    return tok.decode(out[0], skip_special_tokens=True)

# Example: Run with greedy decoding (temperature=0.0)
# response_greedy = chat("What is the primary treatment for strep throat?", temperature=0.0)

# Example: Run with sampling (temperature=0.7)
# response_creative = chat("Write a short poem about a doctor's morning rounds.", temperature=0.7)

In [73]:
import pandas as pd
def create_payload(patient_id:str, user_q:str, k=5, max_tokens=1000, model=LLM_MODEL):
    """For a patient id, and a user question, returns
    Sql, df, non-empathy-optimized summary, optimized summary, optimized score
    returns a dict."""
    sql, rows = answer_patient_question(user_q, patient_id, k, max_tokens)
    df = pd.DataFrame(rows)
    non_optimized_response = summarize_df_with_llm(df, patient_id=patient_id, model=LLM_MODEL)

    optimized_result = optimize_patient_response(
          query=user_q,
          data=df,
          max_retries=3 # Will stop after 3 attempts, even if score < 4
          )

    optimized_emp_score = optimized_result['empathy_score']
    optimized_response = optimized_result['response']

    dict = {
        "sql": sql,
        "df" : df.to_markdown(),
        "non_opt_resp": non_optimized_response,
        "opt_score": optimized_emp_score,
        "opt_response": optimized_response

    }

    return dict


In [74]:
#test
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'which medication am I taking?'
payload = create_payload(patient_id,user_q, model=LLM_MODEL)

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 5/5

*** ROUTE: END (Score > 3) ***


In [75]:
payload.keys()

dict_keys(['sql', 'df', 'non_opt_resp', 'opt_score', 'opt_response'])

In [76]:
payload

{'sql': 'SELECT DISTINCT ON (mr.patient_id, mr.med_name)\n  mr.patient_id,\n  mr.med_name AS medication,\n  mr.dose,\n  mr.route,\n  mr.start_datetime,\n  mr.end_datetime,\n  mr.refills\nFROM medication_requests mr\nWHERE mr.patient_id = :patient_id\n  AND (mr.end_datetime IS NULL OR mr.end_datetime >= NOW())\nORDER BY mr.patient_id,\n         mr.med_name,\n         COALESCE(mr.end_datetime, mr.start_datetime) DESC NULLS LAST;',
 'df': '|    | patient_id                           | medication                                                      |   dose | route   | start_datetime   | end_datetime   | refills   |\n|---:|:-------------------------------------|:----------------------------------------------------------------|-------:|:--------|:-----------------|:---------------|:----------|\n|  0 | 0fca905f-391c-08d3-4b93-b53f69b9da53 | Acetaminophen 325 MG Oral Tablet [Tylenol]                      |      1 |         |                  |                |           |\n|  1 | 0fca905f-391

In [77]:
user_question = user_q
df = payload['df']
portal_response = payload['opt_response']

In [78]:
# ---------- Relevance ----------
judge_prompt_R = """
You are an expert judge that evaluates the quality of a response provided by a patient question-answering LLM system to a patient's query.
Your task is to provide a Relevance score (1–5) for how well the system's response answers the patient's question about their own medical history or records.

***Criterion Definition***
RELEVANCE: The degree to which the response directly and appropriately addresses the specific patient question about their medical record, history, medications, lab results, appointments, or related data.

***Score Definitions and Examples***

1: NOT RELEVANT
- Meaning: The response does not relate to the patient's question.
- Example:
  Q: "What medications am I currently taking?"
  A: "You should eat a balanced diet."
  → Not relevant.

2: SLIGHTLY RELEVANT
- Meaning: The response touches the general topic but not the patient's record.
- Example:
  Q: "When was my last blood test?"
  A: "Blood tests are usually done annually."
  → Only generic information.

3: MODERATELY RELEVANT
- Meaning: The response partially answers or hints at relevant data but misses key specifics.
- Example:
  Q: "What medications am I currently taking?"
  A: "You are prescribed metformin, but I'm not sure about your other medications."
  → Partial relevance.

4: HIGHLY RELEVANT
- Meaning: The response clearly addresses the question using mostly correct patient-specific data, minor omissions acceptable.
- Example:
  Q: "When was my last lab test?"
  A: "Your most recent lab test was in March 2024 for blood glucose."
  → Very good relevance.

5: PERFECTLY RELEVANT
- Meaning: The response is direct, complete, and fully focused on the patient's question and personal data.
- Example:
  Q: "What medications am I currently taking?"
  A: "You are currently taking Metformin 500 mg twice daily and Lisinopril 10 mg once daily."
  → Excellent relevance.

Now evaluate the following pair:

User Question: {user_question}
Portal Response: {portal_response}

***Output Format***
Provide your output strictly as a JSON object:

{{
  "User Question": "{user_question}",
  "Relevance Score": <integer between 1 and 5>
}}

ONLY output the JSON object. Ensure all string values are on a single line and properly escaped.

Do not include explanations, reasoning, or any text outside this JSON object.
"""

# ---------- Faithfulness ----------
judge_prompt_F = """
You are an expert medical data evaluator assessing the factual correctness of a patient-portal system's response.

Your task is to provide a Faithfulness score (1–5) indicating how accurately the response reflects the factual information in the retrieved records.

***Criterion Definition***
FAITHFULNESS: The degree to which the response content is factually consistent with the evidence provided in the retrieved records below.
Do NOT judge relevance or helpfulness — only whether the statements are supported by the retrieved data.

***Score Definitions (with Domain-Relevant Examples)***

1 — NOT FAITHFUL
- Meaning: Mostly incorrect or hallucinated; contradicts the records.
- Example Evidence (Medications):
  | medication | dose  | freq |
  |------------|-------|------|
  | Metformin  | 500mg | BID  |
- Example Response: "You’re taking Metformin 500 mg BID and Atorvastatin 20 mg QD."
- Why: Atorvastatin is not in evidence → contradiction.

2 — SLIGHTLY FAITHFUL
- Meaning: Some correct facts, but significant unsupported claims remain.
- Example Evidence (Labs):
  | test | date       | result |
  |------|------------|--------|
  | A1C  | 2024-03-10 | 6.9%   |
- Example Response: "Your latest A1C on 2024-03-10 was 6.9%, and your LDL was 85 mg/dL."
- Why: A1C detail is correct; LDL is unsupported → major error persists.

3 — MODERATELY FAITHFUL
- Meaning: Mostly supported, but one or two factual inaccuracies (e.g., wrong dose/date).
- Example Evidence (Medications):
  | medication | dose | freq |
  |------------|------|------|
  | Lisinopril | 10mg | QD   |
- Example Response: "You take Lisinopril 5 mg once daily."
- Why: Correct med and frequency; dose is wrong → partial mismatch.

4 — HIGHLY FAITHFUL
- Meaning: Essentially correct; only minor discrepancy/omission (e.g., missing a less salient detail).
- Example Evidence (Appointments):
  | type             | date       | status    |
  |------------------|------------|-----------|
  | Cardio follow-up | 2025-06-14 | completed |
- Example Response: "Your last cardiology follow-up was in June 2025."
- Why: Month/year correct; missing exact day → minor omission.

5 — PERFECTLY FAITHFUL
- Meaning: Fully supported; no contradictions or unsupported claims.
- Example Evidence (Medications):
  | medication | dose  | freq |
  |------------|-------|------|
  | Metformin  | 500mg | BID  |
  | Lisinopril | 10mg  | QD   |
- Example Response: "You are taking Metformin 500 mg twice daily and Lisinopril 10 mg once daily."
- Why: Every stated fact matches the records exactly.

***Retrieved Records***
```markdown
{df_as_text}
Response to Evaluate
{portal_response}
Output Format
Return ONLY the following JSON object (no extra text):

{{
"User Question": "{user_question}",
"Faithfulness Score": <integer between 1 and 5>
}}

ONLY output the JSON object. Ensure all string values are on a single line and properly escaped.

Judging Rules

Use ONLY the provided records for verification; do not infer or assume missing facts.
Formatting differences (e.g., "twice daily" vs "BID") are acceptable if semantically identical.
If no evidence supports a claimed fact, treat it as unsupported.
"""
#---------- Helpfulness ----------
judge_prompt_H = """
You are an expert medical communication evaluator. Judge how HELPFUL the patient-portal system's response is to the patient.
Your task is to provide a Helpfulness score (1–5) indicating whether the response meaningfully helps the patient act on or understand their own medical record question.

Criterion Definition
HELPFULNESS: The degree to which the response is immediately useful to the patient—i.e., it answers the question clearly and provides actionable guidance (when appropriate), next steps, or pointers to where the patient can find the information in their portal. Do NOT judge factual correctness (that's Faithfulness) or topicality (that's Relevance).

Score Definitions (with Patient-Portal Examples)

1 — NOT HELPFUL

Meaning: Off-topic, evasive, or gives no usable guidance.
Example Q: "When was my last cholesterol test?"
Example A: "Cholesterol is a type of fat in the blood."
Why: No answer, no steps.
2 — SLIGHTLY HELPFUL
Meaning: Some topical info but still not actionable or patient-focused.
Example Q: "What meds am I currently taking?"
Example A: "Medications are often used to manage chronic conditions."
Why: General info; provides no path to the patient's own info.
3 — MODERATELY HELPFUL
Meaning: Partially useful; gives a hint or partial answer but lacks clarity or next steps.
Example Q: "Do I have any upcoming appointments?"
Example A: "You can check upcoming visits in the Appointments section."
Why: Direction is given, but no direct status or concrete steps (e.g., exact path).
4 — HIGHLY HELPFUL
Meaning: Clear, patient-centered answer with minor omissions; includes concise steps or references.
Example Q: "Where can I see my vaccine history?"
Example A: "Open Health Records → Immunizations to view your vaccine history. If you don't see recent entries, message your care team from Messages → New Message."
Why: Concrete portal path and next step.
5 — PERFECTLY HELPFUL
Meaning: Direct, concise answer tailored to the question, with clear steps and guardrails when appropriate.
Example Q: "How can I refill my Metformin?"
Example A: "Go to Medications → Metformin → Refill. If 'Refill' isn't available, use Messages → Pharmacy Request or call the number on your prescription label. If you're out of doses today, contact your pharmacy for an emergency supply."
Why: Immediate action path, fallback, and safety guidance.
Judging Rules
Focus ONLY on usefulness to the patient: clarity, directness, actionable steps, and appropriate guardrails.
Prefer concise, structured guidance (e.g., portal navigation paths, who to contact, what to click).
Do not penalize for not repeating data that belongs in the chart; penalize if it fails to guide the patient to it.
Don't reward unnecessary medical theory/jargon if it doesn't help the patient do something.
If the response can't answer due to missing data, helpfulness increases when it explicitly states that and gives exact next steps to obtain it (e.g., where to look or whom to message).
Safety: Encourage contacting clinicians for urgent symptoms; do not judge medical accuracy here.
Input Pair to Evaluate
User Question:
{user_question}
Portal Response:
{portal_response}

Output Format
Return ONLY the following JSON object (no extra text):

{{
"User Question": "{user_question}",
"Helpfulness Score": <integer between 1 and 5>
}}

ONLY output the JSON object. Ensure all string values are on a single line and properly escaped.
"""

In [79]:
# Run medGemma

import json
import re

import json
import re

def safe_json(s: str) -> dict:
    """
    Safely extracts and parses a JSON object from an LLM's string output,
    handling common control character errors, isolation issues, and syntax errors.
    """
    s = s.strip()

    # 1. Escape unescaped control characters.
    # We escape newlines, tabs, and carriage returns.
    s = s.replace('\n', '\\n').replace('\t', '\\t').replace('\r', '\\r')

    # 2. Use a more targeted regex to find the JSON block.
    # This specifically looks for the object structure.
    # re.DOTALL (re.S) ensures '.' matches newlines inside the object.
    m = re.search(r"(\{.*\}|\[.*\])", s, re.DOTALL)

    if m:
        json_str = m.group(0)

        # 3. CRITICAL FIX: Replace unescaped single quotes with double quotes.
        # This solves the most common LLM syntax error: {'key': 'value'} vs {"key": "value"}
        # This uses an aggressive replacement which works for simple strings, but fails on complex nested JSON where
        # a single quote might be a valid character. The most robust solution is often a custom JSON library like `json5`.
        json_str = json_str.replace("'", '"')

        try:
            # 4. Attempt to load the cleaned and isolated JSON string
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            # If standard parsing still fails, fall back to logging the error
            return {"raw": s, "error": str(e)}
    else:
        # 5. No JSON structure found
        return {"raw": s, "error": "No JSON object found."}

def run_judge_prompts(user_question, portal_response, df_as_text):
    """Run Relevance, Faithfulness, and Helpfulness prompts on MedGemma."""

    # ---- PROMPT 1: Relevance ----
    prompt_R = judge_prompt_R.format(user_question=user_question,
                                     portal_response=portal_response)
    reply_R = chat(prompt_R)

    # ---- PROMPT 2: Faithfulness ----
    prompt_F = judge_prompt_F.format(user_question=user_question,
                                     portal_response=portal_response,
                                     df_as_text=df_as_text)
    reply_F = chat(prompt_F)

    # ---- PROMPT 3: Helpfulness ----
    prompt_H = judge_prompt_H.format(user_question=user_question,
                                     portal_response=portal_response)
    reply_H = chat(prompt_H)

    # Helper to extract JSON safely

    result = {
        "Relevance": safe_json(reply_R),
        "Faithfulness": safe_json(reply_F),
        "Helpfulness": safe_json(reply_H)
    }
    return result


#Run MedGemma

In [80]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'which medication am I taking?'
payload = create_payload(patient_id,user_q, model=LLM_MODEL)
user_question = user_q
df = payload['df']
portal_response = payload['opt_response']

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---
-> Score: 5/5

*** ROUTE: END (Score > 3) ***


In [132]:
payload.keys()

dict_keys(['sql', 'df', 'non_opt_resp', 'opt_score', 'opt_response'])

In [81]:
rfh_responses = run_judge_prompts(user_question, portal_response, df_as_text=df)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


In [82]:
rfh_responses.keys()

dict_keys(['Relevance', 'Faithfulness', 'Helpfulness'])

In [83]:
rfh_responses['Relevance']['error']

'Expecting property name enclosed in double quotes: line 1 column 2 (char 1)'

In [84]:
rfh_responses['Relevance']

{'raw': 'You are an expert judge that evaluates the quality of a response provided by a patient question-answering LLM system to a patient\'s query.\\nYour task is to provide a Relevance score (1–5) for how well the system\'s response answers the patient\'s question about their own medical history or records.\\n\\n***Criterion Definition***\\nRELEVANCE: The degree to which the response directly and appropriately addresses the specific patient question about their medical record, history, medications, lab results, appointments, or related data.\\n\\n***Score Definitions and Examples***\\n\\n1: NOT RELEVANT\\n- Meaning: The response does not relate to the patient\'s question.\\n- Example:\\n  Q: "What medications am I currently taking?"\\n  A: "You should eat a balanced diet."\\n  → Not relevant.\\n\\n2: SLIGHTLY RELEVANT\\n- Meaning: The response touches the general topic but not the patient\'s record.\\n- Example:\\n  Q: "When was my last blood test?"\\n  A: "Blood tests are usually do

In [85]:
rfh_responses.keys()

dict_keys(['Relevance', 'Faithfulness', 'Helpfulness'])

In [86]:
print(rfh_responses['Relevance']['raw'])

You are an expert judge that evaluates the quality of a response provided by a patient question-answering LLM system to a patient's query.\nYour task is to provide a Relevance score (1–5) for how well the system's response answers the patient's question about their own medical history or records.\n\n***Criterion Definition***\nRELEVANCE: The degree to which the response directly and appropriately addresses the specific patient question about their medical record, history, medications, lab results, appointments, or related data.\n\n***Score Definitions and Examples***\n\n1: NOT RELEVANT\n- Meaning: The response does not relate to the patient's question.\n- Example:\n  Q: "What medications am I currently taking?"\n  A: "You should eat a balanced diet."\n  → Not relevant.\n\n2: SLIGHTLY RELEVANT\n- Meaning: The response touches the general topic but not the patient's record.\n- Example:\n  Q: "When was my last blood test?"\n  A: "Blood tests are usually done annually."\n  → Only generic i

In [87]:
s = rfh_responses['Relevance']['raw']


In [88]:
import pprint

pprint.pprint(rfh_responses['Relevance'])

{'error': 'Expecting property name enclosed in double quotes: line 1 column 2 '
          '(char 1)',
 'raw': 'You are an expert judge that evaluates the quality of a response '
        "provided by a patient question-answering LLM system to a patient's "
        'query.\\nYour task is to provide a Relevance score (1–5) for how well '
        "the system's response answers the patient's question about their own "
        'medical history or records.\\n\\n***Criterion '
        'Definition***\\nRELEVANCE: The degree to which the response directly '
        'and appropriately addresses the specific patient question about their '
        'medical record, history, medications, lab results, appointments, or '
        'related data.\\n\\n***Score Definitions and Examples***\\n\\n1: NOT '
        "RELEVANT\\n- Meaning: The response does not relate to the patient's "
        'question.\\n- Example:\\n  Q: "What medications am I currently '
        'taking?"\\n  A: "You should eat a balanced di

# MedGemma did not follow JSON output instructions, so we piped its reponse to 4 o mini to generate a clear output

In [89]:
from pydantic import BaseModel, Field
from typing import Literal

class JudgeOutput_R(BaseModel):
    """The final, clean, structured output from the evaluation process."""
    user_question: str = Field(description="The original question from the patient.")
    portal_response: str = Field(description="The full response provided by the patient-facing system.")
    relevance_score: Literal[1, 2, 3, 4, 5] = Field(description="The final Relevance score (1-5) based on the criteria provided in the prompt.")

class JudgeOutput_F(BaseModel):
    """The final, clean, structured output from the evaluation process."""
    user_question: str = Field(description="The original question from the patient.")
    portal_response: str = Field(description="The full response provided by the patient-facing system.")
    faithfulness_score: Literal[1, 2, 3, 4, 5] = Field(description="The final Faithfulness score (1-5) based on the criteria provided in the prompt.")

class JudgeOutput_H(BaseModel):
    """The final, clean, structured output from the evaluation process."""
    user_question: str = Field(description="The original question from the patient.")
    portal_response: str = Field(description="The full response provided by the patient-facing system.")
    helpfulness_score: Literal[1, 2, 3, 4, 5] = Field(description="The final Helpfulness score (1-5) based on the criteria provided in the prompt.")




In [127]:
def get_outputJSON_R(rfh_responses, model, JudgeOutput_R):
  """returns a well formatted json response (user_question, portal_response, relevance score)
  for 'Relevance' key in unprocessed rfh_responses. Uses model = gpt-40-mini
  Expects Pydantic schema JudgeOutput_R
  """

  # Get the specific formatting instructions for GPT-4o mini
  parser_R = PydanticOutputParser(pydantic_object=JudgeOutput_R)
  format_instructions_R = parser_R.get_format_instructions()

  correction_prompt_R = ChatPromptTemplate.from_messages([
      ("system",
      "You are a strict JSON formatter. Your task is to extract the 'User Question', 'Portal Response', and the 'Relevance Score' from the RAW TEXT provided below, and reformat it STRICTLY according to the given JSON schema. Output your response as a JSON object."),
      ("human",
      "RAW TEXT (May contain errors, markdown, or extra text):\n---\n{raw_medgemma_output}\n---\n\n{format_instructions_R}") # <-- Still requires this variable
  ])

  correction_prompt_R_fixed = correction_prompt_R.partial(
      format_instructions_R=format_instructions_R
  )
  correction_chain_R = correction_prompt_R_fixed | model | parser_R

  reply_R = rfh_responses['Relevance']['raw']
  validated_output_R = correction_chain_R.invoke({"raw_medgemma_output": reply_R})
  json_string_R = validated_output_R.model_dump_json(indent=4)

  return json_string_R


def get_outputJSON_F(rfh_responses, model, JudgeOutput_F):
  """returns a well formatted json response (user_question, portal_response, faithfulness score)
  for 'Faithfulness' key in unprocessed rfh_responses. Uses model = gpt-40-mini
  Expects Pydantic schema JudgeOutput_F
  """

  # Get the specific formatting instructions for GPT-4o mini
  # 2. Get the specific formatting instructions for GPT-4o mini
  parser_F = PydanticOutputParser(pydantic_object=JudgeOutput_F)
  format_instructions_F = parser_F.get_format_instructions()

  correction_prompt_F = ChatPromptTemplate.from_messages([
      ("system",
      "You are a strict JSON formatter. Your task is to extract the 'User Question', 'Portal Response', and the 'Faithfulness Score' from the RAW TEXT provided below, and reformat it STRICTLY according to the given JSON schema. Output your response as a JSON object."),
      ("human",
      "RAW TEXT (May contain errors, markdown, or extra text):\n---\n{raw_medgemma_output}\n---\n\n{format_instructions_F}") # <-- Still requires this variable
  ])

  correction_prompt_F_fixed = correction_prompt_F.partial(
      format_instructions_F=format_instructions_F
  )
  correction_chain_F = correction_prompt_F_fixed | model | parser_F

  reply_F = rfh_responses['Faithfulness']['raw']
  validated_output_F = correction_chain_F.invoke({"raw_medgemma_output": reply_F})
  json_string_F = validated_output_F.model_dump_json(indent=4)

  return json_string_F


def get_outputJSON_H(rfh_responses, model, JudgeOutput_H):
  """returns a well formatted json response (user_question, portal_response, helpfulness score)
  for 'Helpfulness' key in unprocessed rfh_responses. Uses model = gpt-40-mini
  Expects Pydantic schema JudgeOutput_H
  """

  # Get the specific formatting instructions for GPT-4o mini
  # 2. Get the specific formatting instructions for GPT-4o mini
  # 2. Get the specific formatting instructions for GPT-4o mini
  parser_H = PydanticOutputParser(pydantic_object=JudgeOutput_H)
  format_instructions_H = parser_H.get_format_instructions()

  correction_prompt_H = ChatPromptTemplate.from_messages([
      ("system",
      "You are a strict JSON formatter. Your task is to extract the 'User Question', 'Portal Response', and the 'Helpfulness Score' from the RAW TEXT provided below, and reformat it STRICTLY according to the given JSON schema. Output your response as a JSON object."),
      ("human",
      "RAW TEXT (May contain errors, markdown, or extra text):\n---\n{raw_medgemma_output}\n---\n\n{format_instructions_H}") # <-- Still requires this variable
  ])

  correction_prompt_H_fixed = correction_prompt_F.partial(
      format_instructions_H=format_instructions_H
  )
  correction_chain_H = correction_prompt_H_fixed | model | parser_H

  reply_H = rfh_responses['Helpfulness']['raw']
  validated_output_H = correction_chain_H.invoke({"raw_medgemma_output": reply_H})
  json_string_H = validated_output_H.model_dump_json(indent=4)

  return json_string_H



In [128]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser

# 1. Instantiate the reliable OpenAI model
gpt_parser_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [130]:
json_R = get_outputJSON_R(rfh_responses=rfh_responses, model=gpt_parser_model, JudgeOutput_R=JudgeOutput_R)
json_F = get_outputJSON_F(rfh_responses=rfh_responses, model=gpt_parser_model, JudgeOutput_F=JudgeOutput_F)
json_H = get_outputJSON_H(rfh_responses=rfh_responses, model=gpt_parser_model, JudgeOutput_H=JudgeOutput_H)
print(json_R)
print(json_F)
print(json_H)

{
    "user_question": "which medication am I taking?",
    "portal_response": "I understand that it can be confusing to keep track of your medications, and I'm here to help you with that. Based on your records, you are currently taking the following medications:\n\n1. **Acetaminophen 325 MG Oral Tablet (Tylenol)**\n2. **Acetaminophen 325 MG / OxyCODONE Hydrochloride 5 MG Oral Tablet**\n3. **Amlodipine 2.5 MG Oral Tablet**\n4. **Naproxen Sodium 220 MG Oral Tablet**\n\nIf you have any questions about these medications, such as their purposes or how to take them, please feel free to ask. It's important to ensure you feel comfortable and informed about your treatment.",
    "relevance_score": 5
}
{
    "user_question": "which medication am I taking?",
    "portal_response": "I understand that it can be confusing to keep track of your medications, and I'm here to help you with that. Based on your records, you are currently taking the following medications:\n\n1. **Acetaminophen 325 MG Oral

In [118]:
parser_R = PydanticOutputParser(pydantic_object=JudgeOutput_R)
format_instructions_R = parser_R.get_format_instructions()
print(format_instructions_R)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "The final, clean, structured output from the evaluation process.", "properties": {"user_question": {"description": "The original question from the patient.", "title": "User Question", "type": "string"}, "portal_response": {"description": "The full response provided by the patient-facing system.", "title": "Portal Response", "type": "string"}, "relevance_score": {"description": "The final Relevance score (1-5) based on the criteria provided in the prompt.", "enum": [1, 2, 3, 4, 5], "title": "Relevance Score", "type": "integer"}

In [104]:
##Faithfulness json extraction from gpt 40 mini

# 2. Get the specific formatting instructions for GPT-4o mini
parser_F = PydanticOutputParser(pydantic_object=JudgeOutput_F)
format_instructions_F = parser_F.get_format_instructions()

correction_prompt_F = ChatPromptTemplate.from_messages([
    ("system",
     "You are a strict JSON formatter. Your task is to extract the 'User Question', 'Portal Response', and the 'Faithfulness Score' from the RAW TEXT provided below, and reformat it STRICTLY according to the given JSON schema. Output your response as a JSON object."),
    ("human",
     "RAW TEXT (May contain errors, markdown, or extra text):\n---\n{raw_medgemma_output}\n---\n\n{format_instructions_F}") # <-- Still requires this variable
])

correction_prompt_F_fixed = correction_prompt_F.partial(
    format_instructions_F=format_instructions_F
)
correction_chain_F = correction_prompt_F_fixed | gpt_parser_model | parser_F


In [105]:
##Helpfulness json extraction from gpt 40 mini

# 2. Get the specific formatting instructions for GPT-4o mini
parser_H = PydanticOutputParser(pydantic_object=JudgeOutput_H)
format_instructions_H = parser_H.get_format_instructions()

correction_prompt_F = ChatPromptTemplate.from_messages([
    ("system",
     "You are a strict JSON formatter. Your task is to extract the 'User Question', 'Portal Response', and the 'Helpfulness Score' from the RAW TEXT provided below, and reformat it STRICTLY according to the given JSON schema. Output your response as a JSON object."),
    ("human",
     "RAW TEXT (May contain errors, markdown, or extra text):\n---\n{raw_medgemma_output}\n---\n\n{format_instructions_H}") # <-- Still requires this variable
])

correction_prompt_H_fixed = correction_prompt_F.partial(
    format_instructions_H=format_instructions_H
)
correction_chain_H = correction_prompt_H_fixed | gpt_parser_model | parser_H

In [106]:
rfh_responses.keys()

dict_keys(['Relevance', 'Faithfulness', 'Helpfulness'])

In [107]:
reply_R = rfh_responses['Relevance']['raw']
reply_F = rfh_responses['Faithfulness']['raw']
reply_H = rfh_responses['Helpfulness']['raw']

In [108]:
validated_output_R = correction_chain_R.invoke({"raw_medgemma_output": reply_R})
validated_output_F = correction_chain_F.invoke({"raw_medgemma_output": reply_F})
validated_output_H = correction_chain_H.invoke({"raw_medgemma_output": reply_H})

In [109]:
json_string_R = validated_output_R.model_dump_json(indent=4)
json_string_F = validated_output_F.model_dump_json(indent=4)
json_string_H = validated_output_H.model_dump_json(indent=4)


In [117]:
print(json_string_R)
print(json_string_F)
print(json_string_H)


{
    "user_question": "which medication am I taking?",
    "portal_response": "I understand that it can be confusing to keep track of your medications, and I'm here to help you with that. Based on your records, you are currently taking the following medications:\n\n1. **Acetaminophen 325 MG Oral Tablet (Tylenol)**\n2. **Acetaminophen 325 MG / OxyCODONE Hydrochloride 5 MG Oral Tablet**\n3. **Amlodipine 2.5 MG Oral Tablet**\n4. **Naproxen Sodium 220 MG Oral Tablet**\n\nIf you have any questions about these medications, such as their purposes or how to take them, please feel free to ask. It's important to ensure you feel comfortable and informed about your treatment.",
    "relevance_score": 5
}


In [155]:
####towards a pipeline

def get_RFH_evaluation(patient_id, user_q):
    """
    for a patient id, user_q, returns RFH JSONs for BOTH
    non optimized (empathy) response, and empathy optimized response,
    and the empathy optimized score for the optimized response
    """
    payload = create_payload(patient_id,user_q, model=LLM_MODEL)
    user_question = user_q
    df = payload['df']
    portal_response = payload['opt_response']

    # payload.keys()
    # dict_keys(['sql', 'df', 'non_opt_resp', 'opt_score', 'opt_response'])

    RFH4_nonoptresponses = run_judge_prompts(user_question, portal_response=payload['non_opt_resp'], df_as_text=df)
    json_R_nonoptresp = get_outputJSON_R(rfh_responses=RFH4_nonoptresponses, model=gpt_parser_model, JudgeOutput_R=JudgeOutput_R)
    json_F_nonoptresp = get_outputJSON_F(rfh_responses=RFH4_nonoptresponses, model=gpt_parser_model, JudgeOutput_F=JudgeOutput_F)
    json_H_nonoptresp = get_outputJSON_H(rfh_responses=RFH4_nonoptresponses, model=gpt_parser_model, JudgeOutput_H=JudgeOutput_H)

    RFH4_optresponses = run_judge_prompts(user_question, portal_response=payload['opt_response'], df_as_text=df)
    json_R_optresp = get_outputJSON_R(rfh_responses=RFH4_optresponses, model=gpt_parser_model, JudgeOutput_R=JudgeOutput_R)
    json_F_optresp = get_outputJSON_F(rfh_responses=RFH4_optresponses, model=gpt_parser_model, JudgeOutput_F=JudgeOutput_F)
    json_H_optresp = get_outputJSON_H(rfh_responses=RFH4_optresponses, model=gpt_parser_model, JudgeOutput_H=JudgeOutput_H)

    return {
        "patient_id": patient_id,
        "user_q": user_q,
        "empathy_opt_score": payload['opt_score'],
        "non_opt_RFH": [json_R_nonoptresp, json_F_nonoptresp, json_H_nonoptresp],
        "opt_RFH": [json_R_optresp, json_F_optresp, json_H_optresp]
    }


In [156]:
patient_id = '0fca905f-391c-08d3-4b93-b53f69b9da53'
user_q = 'which medication am I taking?'
dict_ = get_RFH_evaluation(patient_id, user_q)

--- STARTING OPTIMIZATION (Max Retries: 3) ---

--- GENERATING RESPONSE (Retry: 0) ---

--- EVALUATING RESPONSE ---


Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


-> Score: 4/5

*** ROUTE: END (Score > 3) ***


Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


In [157]:
dict_.keys()

dict_keys(['patient_id', 'user_q', 'empathy_opt_score', 'non_opt_RFH', 'opt_RFH'])

In [165]:
json_string = dict_['non_opt_RFH'][0]
python_dict = json.loads(json_string)
python_dict

{'user_question': 'which medication am I taking?',
 'portal_response': 'The table shows medication records for patient ID 0fca905f-391c-08d3-4b93-b53f69b9da53, listing five entries. All medications are oral tablets, with dosages provided for Acetaminophen and amLODIPine, while the dosage for Naproxen sodium is missing. Notably, there are no start or end dates, and the refills field is also empty for all entries. The table appears truncated, showing only 5 rows.',
 'relevance_score': 4}

In [190]:
def extract_my_dict(dict_):

  d = {
    'patient_id': dict_['patient_id'],
    'user_question' : dict_['user_q'],
    'empathy_opt_score' : dict_['empathy_opt_score'],

    'nonopt_R_score'  : json.loads(dict_['non_opt_RFH'][0])['relevance_score'],
    'nonopt_F_score'  :  json.loads(dict_['non_opt_RFH'][1])['faithfulness_score'],
    'nonopt_H_score'  :  json.loads(dict_['non_opt_RFH'][2])['helpfulness_score'],

    'opt_R_score' :  json.loads(dict_['opt_RFH'][0])['relevance_score'],
    'opt_F_score' :  json.loads(dict_['opt_RFH'][1])['faithfulness_score'],
    'opt_H_score' :  json.loads(dict_['opt_RFH'][2])['helpfulness_score'],

    'nonopt_resp'   :  json.loads(dict_['non_opt_RFH'][0])['portal_response'],
    'opt_resp'   :  json.loads(dict_['opt_RFH'][0])['portal_response']

  }
  return d

In [191]:
print(extract_my_dict(dict_))

{'patient_id': '0fca905f-391c-08d3-4b93-b53f69b9da53', 'user_question': 'which medication am I taking?', 'empathy_opt_score': 4, 'nonopt_R_score': 4, 'nonopt_F_score': 5, 'nonopt_H_score': 2, 'opt_R_score': 5, 'opt_F_score': 5, 'opt_H_score': 5, 'nonopt_resp': 'The table shows medication records for patient ID 0fca905f-391c-08d3-4b93-b53f69b9da53, listing five entries. All medications are oral tablets, with dosages provided for Acetaminophen and amLODIPine, while the dosage for Naproxen sodium is missing. Notably, there are no start or end dates, and the refills field is also empty for all entries. The table appears truncated, showing only 5 rows.', 'opt_resp': "I understand that it can be a bit confusing to keep track of your medications, and I'm here to help you with that. Based on your records, you are currently taking the following medications:\n\n1. **Acetaminophen 325 MG Oral Tablet (Tylenol)**\n2. **Acetaminophen 325 MG / OxyCODONE Hydrochloride 5 MG Oral Tablet**\n3. **Amlodipi

# This is the master file that

takes in a patient id, user question,
returns

*   creates a payload of generate sql, returned df, nonoptimized response, and then empathy optimized response
    payload = create_payload(patient_id,user_q, model=LLM_MODEL)
    payload.keys()
    dict_keys(['sql', 'df', 'non_opt_resp', 'opt_score', 'opt_response'])
    
*   then calcuates RFH scores of BOTH non optimized and optimized responses using MedGemma, and uses gpt 4o mini to format outputs in a JSON

**Pipeline functions**

**def get_RFH_evaluation(patient_id, user_q):**


    RFH4_nonoptresponses = run_judge_prompts(user_question, portal_response=payload['non_opt_resp'], df_as_text=df)
    json_R_nonoptresp = get_outputJSON_R(rfh_responses=RFH4_nonoptresponses, model=gpt_parser_model, JudgeOutput_R=JudgeOutput_R)
    json_F_nonoptresp = get_outputJSON_F(rfh_responses=RFH4_nonoptresponses, model=gpt_parser_model, JudgeOutput_F=JudgeOutput_F)
    json_H_nonoptresp = get_outputJSON_H(rfh_responses=RFH4_nonoptresponses, model=gpt_parser_model, JudgeOutput_H=JudgeOutput_H)

    RFH4_optresponses = run_judge_prompts(user_question, portal_response=payload['opt_response'], df_as_text=df)
    json_R_optresp = get_outputJSON_R(rfh_responses=RFH4_optresponses, model=gpt_parser_model, JudgeOutput_R=JudgeOutput_R)
    json_F_optresp = get_outputJSON_F(rfh_responses=RFH4_optresponses, model=gpt_parser_model, JudgeOutput_F=JudgeOutput_F)
    json_H_optresp = get_outputJSON_H(rfh_responses=RFH4_optresponses, model=gpt_parser_model, JudgeOutput_H=JudgeOutput_H)

    return {
        "patient_id": patient_id,
        "user_q": user_q,
        "empathy_opt_score": payload['opt_score'],
        "non_opt_RFH": [json_R_nonoptresp, json_F_nonoptresp, json_H_nonoptresp],
        "opt_RFH": [json_R_optresp, json_F_optresp, json_H_optresp]
    }



**then use extract_my_dict(dict_) to get:**

    {
      'patient_id': '0fca905f-391c-08d3-4b93-b53f69b9da53',
      'user_question': 'which medication am I taking?',
      'empathy_opt_score': 4,
      
      'nonopt_R_score': 4,
      'nonopt_F_score': 5,
      'nonopt_H_score': 2,
      
      'opt_R_score': 5,
      'opt_F_score': 5,
      'opt_H_score': 5,
      
      'nonopt_resp':
          'The table shows medication records for patient ID 0fca905f-391c-08d3-4b93-b53f69b9da53, listing five entries. All medications are oral tablets, with dosages provided for Acetaminophen and amLODIPine, while the dosage for Naproxen sodium is missing. Notably, there are no start or end dates, and the refills field is also empty for all entries. The table appears truncated, showing only 5 rows.',
      
      'opt_resp':
          "I understand that it can be a bit confusing to keep track of your medications, and I'm here to help you with that. Based on your records, you are currently taking the following medications:\n\n1. **Acetaminophen 325 MG Oral Tablet (Tylenol)**\n2. **Acetaminophen 325 MG / OxyCODONE Hydrochloride 5 MG Oral Tablet**\n3. **Amlodipine 2.5 MG Oral Tablet**\n4. **Naproxen Sodium 220 MG Oral Tablet**\n\nIf you have any questions about these medications, such as their purposes or how to take them, please feel free to ask. It's important to feel comfortable and informed about your treatment."

    }


